# Inspect data


In [ ]:
import numpy as np
import json
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.mapping.map_utils import get_geometric_score, get_rocks_score
from lac.utils.plotting import (
    plot_poses,
    plot_3d_points,
    plot_surface,
    plot_heightmaps,
    plot_rock_map,
    plot_rock_maps,
    plot_rock_results,
    plot_height_error,
    plot_loop_closures,
)
from lac.util import load_data
from lac.params import DT

%load_ext autoreload
%autoreload 2

### Load data


In [ ]:
# data_path = "../../output/DataCollectionAgent/double_loop_preset1"
# data_path = "/home/shared/data_raw/LAC/runs/triangles_preset6"
data_path = "../../output/NavAgent/default_run"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

### Inspect poses


In [ ]:
slam_poses = np.load(f"{data_path}/slam_poses.npy")

In [ ]:
fig = plot_poses(poses, color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(slam_poses, fig=fig, color="orange", no_axes=True, name="SLAM")
fig.show()

In [ ]:
backend_state = np.load(f"{data_path}/backend_state.npz")

In [ ]:
fig = plot_poses(poses, color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(slam_poses, fig=fig, color="orange", no_axes=True, name="SLAM")
fig = plot_loop_closures(slam_poses, backend_state["loop_closures"], fig=fig, showlegend=False)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
slam_velocities = np.linalg.norm(np.diff(slam_poses[:, :3, 3], axis=0) / DT, axis=1)

In [ ]:
poses = np.array(poses)
gt_velocities = np.linalg.norm(np.diff(poses[:, :3, 3], axis=0) / DT, axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=2 * np.arange(len(slam_velocities)) + 80,
        y=slam_velocities / 2,
        mode="lines",
        name="SLAM Velocities",
        line=dict(color="orange"),
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(len(gt_velocities)),
        y=gt_velocities,
        mode="lines",
        name="GT Velocities",
        line=dict(color="black"),
    )
)
fig.update_layout(
    title="SLAM vs GT Velocities",
    xaxis_title="Time Step",
    yaxis_title="Velocity (m/s)",
)
fig.show()

In [ ]:
json_data = json.load(open(f"{data_path}/data_log.json"))

commanded_vs = []

for frame in json_data["frames"]:
    commanded_vs.append(frame["control"]["v"])

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(len(commanded_vs)),
        y=commanded_vs,
        mode="lines",
        name="commanded Velocities",
        line=dict(color="black"),
    )
)
fig.show()

### Inspect maps


In [ ]:
ground_truth_map = np.load(Path(data_path) / "Moon_Map_01_6_rep0.dat", allow_pickle=True)
agent_map = np.load(Path(data_path) / "Moon_Map_01_6_rep0_agent.dat", allow_pickle=True)

In [ ]:
plot_height_error(ground_truth_map, agent_map)

In [ ]:
plot_heightmaps(ground_truth_map, agent_map)

In [ ]:
plot_surface(agent_map)

In [ ]:
plot_rock_maps(ground_truth_map, agent_map)

In [ ]:
plot_rock_results(ground_truth_map, agent_map)

In [ ]:
print(f"Geometric score: {get_geometric_score(ground_truth_map, agent_map)}")
print(f"Rocks score: {get_rocks_score(ground_truth_map, agent_map)}")

In [ ]:
plot_rock_map(ground_truth_map)

In [ ]:
plot_rock_map(agent_map)

In [ ]:
ground_truth_map_points = ground_truth_map[:, :, :3].reshape(-1, 3)
agent_map_points = agent_map[:, :, :3].reshape(-1, 3)

fig = plot_3d_points(ground_truth_map_points, color="blue", name="Ground Truth")
fig = plot_3d_points(agent_map_points, fig=fig, color="red", name="Agent Map")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Inspect rock points


In [ ]:
data_path = "../../output/nav_agent"
rock_points = np.load(f"{data_path}/rock_points.npy")

In [ ]:
fig = plot_3d_points(rock_points)
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
import os

cams_geoms = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))[
    "rover"
]["cameras"]
cams_geoms["FrontLeft"]